In [1]:
!git clone https://github.com/tensorflow-project/FineTuning

Cloning into 'FineTuning'...
remote: Enumerating objects: 1366, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (91/91), done.
remote: Total 1366 (delta 51), reused 0 (delta 0), pack-reused 1272
Receiving objects: 100% (1366/1366), 156.38 MiB | 30.32 MiB/s, done.
Resolving deltas: 100% (999/999), done.


In [2]:
import tensorflow as tf
import numpy as np
import datetime
import sys
import os

py_file_location = "/content/FineTuning/Classification"
sys.path.append(os.path.abspath(py_file_location))
py_file_location = "/content/FineTuning/models"
sys.path.append(os.path.abspath(py_file_location))

from resnet import ResNet
import preprocessing as pre

In [3]:
!wget <https://github.com/tensorflow-project/FineTuning/blob/main/dataset/happyZip.zip>
!wget <https://github.com/tensorflow-project/FineTuning/blob/main/dataset/loveZip.zip>
!wget <https://github.com/tensorflow-project/FineTuning/blob/main/dataset/angryZip.zip>
!wget <https://github.com/tensorflow-project/FineTuning/blob/main/dataset/sadZip.zip>

!unzip /content/FineTuning/dataset/happyZip.zip -d /content/FineTuning/dataset/happyZip
!unzip /content/FineTuning/dataset/angryZip.zip -d /content/FineTuning/dataset/angryZip
!unzip /content/FineTuning/dataset/sadZip.zip -d /content/FineTuning/dataset/sadZip
!unzip /content/FineTuning/dataset/loveZip.zip -d /content/FineTuning/dataset/loveZip

path_angry_train = '/content/FineTuning/dataset/angryZip/angry/train/'
path_angry_test = '/content/FineTuning/dataset/angryZip/angry/test/'
path_sad_train = '/content/FineTuning/dataset/sadZip/sad/train/'
path_sad_test = '/content/FineTuning/dataset/sadZip/sad/test/'
path_love_train = '/content/FineTuning/dataset/loveZip/love/train/'
path_love_test = '/content/FineTuning/dataset/loveZip/love/test/'
path_happy_train = '/content/FineTuning/dataset/happyZip/happy/train/'
path_happy_test = '/content/FineTuning/dataset/happyZip/happy/test/'

/bin/bash: -c: line 0: syntax error near unexpected token `newline'
/bin/bash: -c: line 0: `wget <https://github.com/tensorflow-project/FineTuning/blob/main/dataset/happyZip.zip>'
/bin/bash: -c: line 0: syntax error near unexpected token `newline'
/bin/bash: -c: line 0: `wget <https://github.com/tensorflow-project/FineTuning/blob/main/dataset/loveZip.zip>'
/bin/bash: -c: line 0: syntax error near unexpected token `newline'
/bin/bash: -c: line 0: `wget <https://github.com/tensorflow-project/FineTuning/blob/main/dataset/angryZip.zip>'
/bin/bash: -c: line 0: syntax error near unexpected token `newline'
/bin/bash: -c: line 0: `wget <https://github.com/tensorflow-project/FineTuning/blob/main/dataset/sadZip.zip>'
Archive:  /content/FineTuning/dataset/happyZip.zip
   creating: /content/FineTuning/dataset/happyZip/happy/new_/
  inflating: /content/FineTuning/dataset/happyZip/happy/test/image_42_happyweights10.jpg  
  inflating: /content/FineTuning/dataset/happyZip/happy/test/image_66_happyweig

In [4]:
%load_ext tensorboard

In [5]:
### build datasets,
angry_train = pre.dataset(path_angry_train, 1, augment=True)
sad_train = pre.dataset(path_sad_train, 2, augment=True)
love_train = pre.dataset(path_love_train, 3, augment=True)
happy_train = pre.dataset(path_happy_train, 4, augment=True)

###concatenate, shuffle, batch, prefetch them
train_ds = (happy_train.concatenate(love_train)).concatenate(sad_train.concatenate(angry_train))
train_ds = train_ds.shuffle(100)
train_ds = train_ds.batch(32)
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)

angry_test = pre.dataset(path_angry_test, 1)
sad_test = pre.dataset(path_sad_test, 2)
love_test = pre.dataset(path_love_test, 3)
happy_test = pre.dataset(path_happy_test, 4)

test_ds = (happy_test.concatenate(love_test)).concatenate(sad_test.concatenate(angry_test))
test_ds = test_ds.shuffle(1000)
test_ds = test_ds.batch(64)
test_ds = test_ds.prefetch(tf.data.AUTOTUNE)

In [6]:
# choose path to save weights
save_path = "trained_model_Run1"

In [7]:
from resnet import ResNet

In [8]:
### Clear any logs from previous runs
!rm -rf logs/fit/*

In [ ]:
### Create new model with custom layers
model = ResNet()

### Compile the model
model.compile(optimizer=model.optimizer , loss='categorical_crossentropy', metrics=['accuracy'])

### Define log directory, logs are created and stored
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

### Train the model
### either with or without Tensorboard
#model.fit(train_ds, validation_data=test_ds, epochs=30)
model.fit(train_ds, validation_data=test_ds, epochs=30, callbacks=[tensorboard_callback])

94765736/94765736 [==============================] - 0s 0us/step
Epoch 1/30
14/26 [===============>..............] - ETA: 1:01 - loss: 0.4953 - accuracy: 0.5357

In [ ]:
### load Tensorboard
%tensorboard --logdir logs/fit